In [ ]:
# Load locations
import pandas as pd
from anyio import sleep

# Load the Excel file and use the first row as headers
file_path = 'data/locations_with_coords.xlsx'  # Update the path as needed
locations = pd.read_excel(file_path, header=0)  # 'header=0' makes the first row the column headers

# Display the header (first few rows)
locations.head()

In [ ]:
# Load routes

file_path = 'data/StrategischerTourenplan_HSLU.xlsx'  
routes = pd.read_excel(file_path, sheet_name=0, header=0)  # 'header=0' makes the first row the column headers

# Display the header (first few rows)
routes.head(10)

In [ ]:
# Remove Bahn
routes = routes[routes['Tourtyp'] != 'Bahn']

In [ ]:
routes_grouped = routes.groupby(['Tournummer','VST']).first().reset_index().__deepcopy__()
routes_grouped.head(10)

In [ ]:
#in locations dataframe replace column name 'VST-Nummer' with 'SAP'
locations.rename(columns={'VST-Nummer':'SAP'}, inplace=True)
locations.head()

In [ ]:
# Find duplicates in the 'locations' dataframe based on the 'SAP' column
duplicate_locations = locations[locations.duplicated(subset='SAP', keep=False)]

# Display the duplicates
print(duplicate_locations)

In [ ]:
# Remove duplicates from the 'locations' dataframe based on the 'SAP' column
locations = locations.drop_duplicates(subset='SAP')

In [ ]:
# Set 'SAP' as the index in both dataframes for alignment
routes_with_address = routes_grouped.join(locations.set_index('SAP'), on='SAP', how='left')

# This way, the original index of routes_grouped is preserved, and no extra rows are added.
routes_with_address.head()

In [ ]:
routes_with_address.to_excel('data/routes_with_address.xlsx')

In [ ]:
import openrouteservice

# Initialize the OpenRouteService client with your API key
API_KEY = 'your-api-key'
client = openrouteservice.Client(key=API_KEY)

# Define the fixed address
base_address = "xxx, yyy, zzz"  # Update with the base address

geocode_fixed = client.pelias_search(base_address)
base_coords = geocode_fixed['features'][0]['geometry']['coordinates']
base_lat, base_lon = base_coords[1], base_coords[0]

In [ ]:
routes_with_address.head()

In [ ]:
# Add base segment after each tour
def add_base_after_tour(group):
    base_row = pd.DataFrame({
        'Tournummer': [group['Tournummer'].iloc[0]],
        'Tagestournummer': [group['Tagestournummer'].iloc[0]],
        'Wochentag': [group['Wochentag'].iloc[0]],
        'VST': 'Schafisheim',
        'lat': base_lat,
        'long': base_lon,
    })
    
    return pd.concat([group, base_row])

# Group by 'Category' and apply the function
routes_with_address = routes_with_address.groupby('Tournummer', group_keys=False).apply(add_base_after_tour).reset_index(drop=True)

routes_with_address.head()

In [ ]:
# Initialize the OpenRouteService client with your API key
client = openrouteservice.Client(base_url='http://localhost:8080/ors')

# Calculate distances and add to dataframe
def calculate_distance(lat1, lon1, lat2, lon2):
    routes = client.directions(
        coordinates=[[lon1, lat1], [lon2, lat2]],
        profile='driving-car',
        format='geojson'
    )
    distance = routes['features'][0]['properties']['segments'][0]['distance']
    duration = routes['features'][0]['properties']['segments'][0]['duration']
    return distance / 1000, duration  # Convert meters to kilometers

In [ ]:
import time
prev_row = None
distance = []
duration = []
for index, row in routes_with_address.iterrows():
    if prev_row is None:
        row_distance, row_duration = calculate_distance(base_lat,base_lon,row['lat'],row['long'])
    else:
        if prev_row['Tournummer'] == row['Tournummer']:
            row_distance, row_duration = calculate_distance(prev_row['lat'],prev_row['long'],row['lat'],row['long'])
        else:
            row_distance, row_duration = calculate_distance(base_lat,base_lon,row['lat'],row['long'])
    distance.append(row_distance)
    duration.append(row_duration)
    prev_row = row
routes_with_address['distance'] = distance
routes_with_address['duration'] = duration

In [ ]:
routes_with_address.to_excel('data/routes_with_distance.xlsx')